# Create Monthly CESM ice flux files

In [1]:
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import datetime
from mpl_toolkits.basemap import Basemap, cm
import cmocean
import matplotlib
import modules.mapping_functions as mf

%matplotlib inline

##### Load files

In [2]:
mesh  = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
tmask = np.array(mesh.variables['tmask'])
tmask = np.array(tmask[0,:,:,:])
mlons = np.array(mesh.variables['nav_lon'])
mlats = np.array(mesh.variables['nav_lat'])
Z_masked = np.ma.masked_where((tmask > 0.1), tmask) 

In [3]:
folder = '/ocean/brogalla/GEOTRACES/data/NCAR/'

faero_001_n = nc.Dataset(f'{folder}merged_faero_ocn001_nh.nc')  #hydrophilic black carbon
faero_002_n = nc.Dataset(f'{folder}merged_faero_ocn002_nh.nc')  #hydrophobic black carbon
faero_003_n = nc.Dataset(f'{folder}merged_faero_ocn003_nh.nc')  #dust

ocn001 = np.array(faero_001_n.variables['faero_ocn001'])
ocn002 = np.array(faero_002_n.variables['faero_ocn002'])
ocn003 = np.array(faero_003_n.variables['faero_ocn003'])
time   = faero_001_n.variables['time']
lons   = np.array(faero_001_n.variables['TLON'])
lats   = np.array(faero_001_n.variables['TLAT'])

In [4]:
for i in range(0,lons.shape[0]):
    for j in range(0,lons.shape[1]):
        if lons[i,j] >= 180:
            lons[i,j] = -360+lons[i,j]

##### Functions

In [5]:
def save_file(filename, field1, field2, field3):
    ncd = nc.Dataset(filename, 'w', zlib=True)
    
    ncd.createDimension('x',len(mesh.dimensions['x']))
    ncd.createDimension('y',len(mesh.dimensions['y']))
    ncd.createDimension('time_counter',None)
    
    # variables
    dust             = ncd.createVariable('dust', 'float64', ('y','x'))
    dust.units       = 'kg/m2 s'
    dust.long_name   = 'Dust deposition flux'  
    dust.coordinates = 'nav_lon nav_lat'
    dust[:]          = field1[:]
    
    bc_philic             = ncd.createVariable('bc_philic', 'float64', ('y','x'))
    bc_philic.units       = 'kg/m2 s'
    bc_philic.long_name   = 'Hydrophilic black carbon'  
    bc_philic.coordinates = 'nav_lon nav_lat'
    bc_philic[:]          = field2[:]
    
    bc_phobic             = ncd.createVariable('bc_phobic', 'float64', ('y','x'))
    bc_phobic.units       = 'kg/m2 s'
    bc_phobic.long_name   = 'Hydrophobic black carbon'  
    bc_phobic.coordinates = 'nav_lon nav_lat'
    bc_phobic[:]          = field3[:]
    
    print('saved ', filename)

    ncd.close()
    return

In [6]:
def find_dates(file_year, time):
    file_date_start = file_year*365
    file_date_end = (file_year+1)*365
    
    start_index = []
    end_index = []

    for i in range(0,len(time)):
        if time[i] == file_date_start:
            start_index = i
        elif time[i] == file_date_end:
            end_index = i

#     print('start index: ', start_index)
#     print('end index: ', end_index)
    
    return start_index, end_index

In [7]:
def data_to_ANHA12(file_year, time, lons, lats, ocn001_masked, ocn002_masked, ocn003_masked, savefiles=False):
    
    start_index, end_index = find_dates(file_year, time)
    
    interp_dst = np.empty((12, 2400, 1632))
    interp_bc1  = np.empty((12, 2400, 1632))
    interp_bc2  = np.empty((12, 2400, 1632))

    # loop over the months:
    for i in range(0,12):
        filt_ocn001 = ocn001_masked[start_index+i,:,:][~ocn001_masked[start_index+i,:,:].mask].data
        filt_ocn002 = ocn002_masked[start_index+i,:,:][~ocn002_masked[start_index+i,:,:].mask].data
        filt_ocn003 = ocn003_masked[start_index+i,:,:][~ocn003_masked[start_index+i,:,:].mask].data
        filt_lons1 = lons[~ocn001_masked[start_index+i,:,:].mask].data
        filt_lons2 = lons[~ocn002_masked[start_index+i,:,:].mask].data
        filt_lons3 = lons[~ocn003_masked[start_index+i,:,:].mask].data
        filt_lats1 = lats[~ocn001_masked[start_index+i,:,:].mask].data
        filt_lats2 = lats[~ocn002_masked[start_index+i,:,:].mask].data
        filt_lats3 = lats[~ocn003_masked[start_index+i,:,:].mask].data

        interp_dst[i,:,:]  = mf.interp_np(filt_lons3, filt_lats3, filt_ocn003, mlons, mlats, flatten=False)
        interp_bc1[i,:,:]  = mf.interp_np(filt_lons1, filt_lats1, filt_ocn001, mlons, mlats, flatten=False)
        interp_bc2[i,:,:]  = mf.interp_np(filt_lons2, filt_lats2, filt_ocn002, mlons, mlats, flatten=False)
        
    if savefiles:
        location='/ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/'
        
        for i in range(1,13):
            save_file(f'{location}ice_flux_y{file_year}m{i:02}.nc',interp_dst[i-1,:,:],\
                      interp_bc1[i-1,:,:], interp_bc2[i-1,:,:])
    
    return interp_dst, interp_bc1, interp_bc2

##### Calculations

In [8]:
ocn001_masked = np.ma.masked_where((ocn001 >= 1e30), ocn001) 
ocn002_masked = np.ma.masked_where((ocn002 >= 1e30), ocn002) 
ocn003_masked = np.ma.masked_where((ocn003 >= 1e30), ocn003) 

In [12]:
for year in np.arange(2015,2021,1):
    print(year)
    interp_dst, interp_bc1, interp_bc2 = data_to_ANHA12(year, time, lons, lats, \
                                                        ocn001_masked, ocn002_masked, ocn003_masked,\
                                                        savefiles=True)

2015
saved  /ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/ice_flux_y2015m01.nc
saved  /ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/ice_flux_y2015m02.nc
saved  /ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/ice_flux_y2015m03.nc
saved  /ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/ice_flux_y2015m04.nc
saved  /ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/ice_flux_y2015m05.nc
saved  /ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/ice_flux_y2015m06.nc
saved  /ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/ice_flux_y2015m07.nc
saved  /ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/ice_flux_y2015m08.nc
saved  /ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/ice_flux_y2015m09.nc
saved  /ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/ice_flux_y2015m10.nc
saved  /ocean/brogalla/GEOTRACES/data/Pb-202205-forcing/atmospheric/ice_flux_y2015m11.nc
saved  /ocean/br